<a href="https://colab.research.google.com/github/vizzies/Building-BERT-Model/blob/master/Semantic_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# from google.colab import drive
# drive.mount(‘/content/gdrive’)

In [1]:
# GPU Setup

import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


# Import Data Below and Parse

In [4]:
import pandas

import unicodedata

with open('/content/arc-code-ti-publications.pkl', 'rb') as f:
    pubs = pandas.read_pickle(f)

import re
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)
    
def preprocess_text(sen):

    sentence = str(sen)

    # Removing html tags
    sentence = remove_tags(sentence)

    # Remove hyphenation if at the end of a line
    sentence = sentence.replace('-\n', '')

    # Fix ligatures
    sentence = unicodedata.normalize("NFKD", sentence)

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

# Not really needed any more but will leave in and just comment out
# full_texts = []
# sentences = list(pubs['Text'])
# for sen in sentences:
#     full_texts.append(preprocess_text(str(sen)))

pubs['Text Processed'] = pubs.apply(lambda row: preprocess_text(row['Text']), axis=1)

text_df = pubs[['Text Processed',]].copy()

print(text_df)


                                          Text Processed
Index                                                   
0      Adaptive Stress Testing of Trajectory Predicti...
1      Capturing Analyzing Requirements with FRET Dim...
2                                     PDF error occurred
3      The Ten Lockheed Martin Cyber Physical Challen...
4      Generation of Formal Requirements from Structu...
...                                                  ...
669    A Flexible Evolvable Architecture for Constell...
670    Extended Abstract General Purpose Data Driven ...
671     PARAMETRIC ANALYSIS OF HOVER TEST VEHICLE USI...
672     Bringing Web to Government Research Case Stud...
673    Online Detection and Modeling of Safety Bounda...

[675 rows x 1 columns]


In [3]:
!pip install -U sentence-transformers

Requirement already up-to-date: sentence-transformers in /usr/local/lib/python3.6/dist-packages (0.3.6)


## BERT Sentence Tranformers Semantic Search

In [22]:
"""
This is a simple application for sentence embeddings: semantic search
We have a corpus with various sentences. Then, for a given query sentence,
we want to find the most similar sentence in this corpus.
This script outputs for various queries the top 5 most similar sentences in the corpus.
"""
from sentence_transformers import SentenceTransformer
import scipy.spatial
import pickle as pkl
embedder = SentenceTransformer('bert-base-nli-mean-tokens')

sentences = list(text_df['Text Processed'])[:100]

# Eaxmple query sentences
queries = ['How to evolve architecture for constellations and simulation', 'Build behavior of complex aerospace and modeling of safety']
query_embeddings = embedder.encode(queries,show_progress_bar=True)
text_embeddings = embedder.encode(sentences, show_progress_bar=True)
#
# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
closest_n = 5
print("\nTop 5 most similar sentences in corpus:")
for query, query_embedding in zip(queries, query_embeddings):
    distances = scipy.spatial.distance.cdist([query_embedding], text_embeddings, "cosine")[0]

    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])

    print("\n\n=========================================================")
    print("==========================Query==============================")
    print("===",query,"=====")
    print("=========================================================")


    for idx, distance in results[0:closest_n]:
        print("Score:   ", "(Score: %.4f)" % (1-distance) , "\n" )
        row_dict = pubs.iloc[idx].to_dict() # pubs.loc[pubs.index== sentences[idx]].to_dict()
        #print(row_dict)
        print("Title:  " , row_dict["Title"]  , "\n")
        print("Abstract:  " , row_dict["Abstract"] , "\n")
        print("-------------------------------------------")



Top 5 most similar sentences in corpus:


==========================Query==============================
=== How to evolve architecture for constellations and simulation =====
Score:    (Score: 0.6640) 

Title:   Time-Varying Modification of Reference Model for Adaptive Control with Performance Optimization 

Abstract:   

Abstract—This paper presents a new adaptive control ap-
proach that involves a performance optimization objective.
The control synthesis involves the design of a performance
optimizing adaptive controller from a subset of control
inputs. The resulting effect of the performance optimizing
adaptive controller is to modify the initial reference model
into a time-varying reference model which satisﬁes the
performance optimization requirement obtained from an
optimal control problem. The time-varying reference model
modiﬁcation is accomplished by the real-time solutions of
the time-varying Riccati and Sylvester equations coupled
with the least-squares parameter estimatio